**Kate Groschner** <br>
**AY250 Homework 6**

## Question 1

**Feature 1**

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import skimage 
from skimage import io
from skimage.color import rgb2gray


In [ ]:
def import_exImg():
    

In [6]:
def avg_red(img):
    if len(img.shape) == 3:
        avg = img[:,0].mean()
    return avg

In [7]:
def avg_blue(img):
    if len(img.shape) == 3:
        avg = img[:,1].mean()
    return avg

In [8]:
def avg_green(img):
    if len(img.shape) == 3:
        avg = img[:,2].mean()
    return avg

In [9]:
# edges from sobel
def sobel_edges(image):
    edges = filters.sobel(image)
    return edges

In [11]:
# number of blobs from difference of gaussian
def blobs_dog(image):
    image_gray = rgb2gray(image)
    blob_dog = blob_dog(image_gray, max_sigma=50, threshold=.1)
    blobs = blob_dog[:,3]
    num_blobs = len(blobs)
    return blobs, num_blobs